In [ ]:
! pip install kaggle

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
#https://www.kaggle.com/code/bettycxh06/rri-amplitudes/input
!kaggle datasets download -d bettycxh06/apnea-ecg-database

!kaggle kernels pull bettycxh06/rri-amplitudes


 99% 312M/315M [00:16<00:00, 22.4MB/s]
100% 315M/315M [00:16<00:00, 19.6MB/s]
Source code downloaded to /content/rri-amplitudes.ipynb


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
os.listdir(os.getcwd())

['.config',
 'rri-amplitudes.ipynb',
 'apnea-ecg-database.zip',
 'kaggle.json',
 'sample_data']

In [ ]:
!unzip apnea-ecg-database.zip

Archive:  apnea-ecg-database.zip
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/ANNOTATORS  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/RECORDS  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/SHA256SUMS.txt  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.apn  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.dat  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.hea  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.qrs  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.xws  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.apn  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.hea  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.qrs  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.xws  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01r.apn  

In [ ]:
!pip install biosppy
!pip install wfdb

from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import posixpath
import csv
import wfdb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 3.2 MB/s eta 0:00:00


In [ ]:
import pickle
import sys
from concurrent.futures import ProcessPoolExecutor, as_completed

import biosppy.signals.tools as st
import numpy as np
import os
import wfdb
from biosppy.signals.ecg import correct_rpeaks, hamilton_segmenter
from scipy.signal import medfilt
from tqdm import tqdm

# PhysioNet Apnea-ECG dataset
# url: https://physionet.org/physiobank/database/apnea-ecg/
#base_dir = "../input/apnea-ecg-database/apnea-ecg-database-1.0.0"
base_dir = "apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0"
#base_dir = "physionet.org/files/apnea-ecg/1.0.0"



fs = 100
sample = fs * 60  # 1 min's sample points

before = 2  # forward interval (min)
after = 2  # backward interval (min)
hr_min = 20
hr_max = 300

num_worker = 35


def worker(name, labels):
    X = []
    y = []
    groups = []
    signals = wfdb.rdrecord(os.path.join(base_dir, name), channels=[0]).p_signal[:, 0]
    for j in tqdm(range(len(labels)), desc=name, file=sys.stdout):
        if j < before or \
                (j + 1 + after) > len(signals) / float(sample):
            continue
        signal = signals[int((j - before) * sample):int((j + 1 + after) * sample)]
        signal, _, _ = st.filter_signal(signal, ftype='FIR', band='bandpass', order=int(0.3 * fs),
                                        frequency=[3, 45], sampling_rate=fs)
        # Find R peaks
        rpeaks, = hamilton_segmenter(signal, sampling_rate=fs)
        rpeaks, = correct_rpeaks(signal, rpeaks=rpeaks, sampling_rate=fs, tol=0.1)
        if len(rpeaks) / (1 + after + before) < 40 or \
                len(rpeaks) / (1 + after + before) > 200:  # Remove abnormal R peaks signal
            continue
        # Extract RRI, Ampl signal
        rri_tm, rri_signal = rpeaks[1:] / float(fs), np.diff(rpeaks) / float(fs)
        rri_signal = medfilt(rri_signal, kernel_size=3)
        ampl_tm, ampl_siganl = rpeaks / float(fs), signal[rpeaks]
        hr = 60 / rri_signal
        # Remove physiologically impossible HR signal
        if np.all(np.logical_and(hr >= hr_min, hr <= hr_max)):
            # Save extracted signal
            X.append([(rri_tm, rri_signal), (ampl_tm, ampl_siganl)])
            y.append(0. if labels[j] == 'N' else 1.)
            groups.append(name)
    return X, y, groups


if __name__ == "__main__":
    apnea_ecg = {}

    names = [
        "a01", "a02", "a03", "a04", "a05", "a06", "a07", "a08", "a09", "a10",
        "a11", "a12", "a13", "a14", "a15", "a16", "a17", "a18", "a19", "a20",
        "b01", "b02", "b03", "b04", "b05",
        "c01", "c02", "c03", "c04", "c05", "c06", "c07", "c08", "c09", "c10"
    ]

    o_train = []
    y_train = []
    groups_train = []
    print('Training...')
    with ProcessPoolExecutor(max_workers=num_worker) as executor:
        task_list = []
        for i in range(len(names)):
            labels = wfdb.rdann(os.path.join(base_dir, names[i]), extension="apn").symbol
            task_list.append(executor.submit(worker, names[i], labels))

        for task in as_completed(task_list):
            X, y, groups = task.result()
            o_train.extend(X)
            y_train.extend(y)
            groups_train.extend(groups)

    print()



    answers = {}
    with open(os.path.join("event-2-answers"), "r") as f:
    #    with open(os.path.join(base_dir, "event-2-answers"), "r") as f:
        for answer in f.read().split("\n\n"):
            answers[answer[:3]] = list("".join(answer.split()[2::2]))

    names = [
        "x01", "x02", "x03", "x04", "x05", "x06", "x07", "x08", "x09", "x10",
        "x11", "x12", "x13", "x14", "x15", "x16", "x17", "x18", "x19", "x20",
        "x21", "x22", "x23", "x24", "x25", "x26", "x27", "x28", "x29", "x30",
        "x31", "x32", "x33", "x34", "x35"
    ]

    o_test = []
    y_test = []
    groups_test = []
    print("Testing...")
    with ProcessPoolExecutor(max_workers=num_worker) as executor:

        task_list = []
        for i in range(len(names)):
            labels = answers[names[i]]
            task_list.append(executor.submit(worker, names[i], labels))

        for task in as_completed(task_list):
            X, y, groups = task.result()
            o_test.extend(X)
            y_test.extend(y)
            groups_test.extend(groups)

    apnea_ecg = dict(o_train=o_train, y_train=y_train, groups_train=groups_train, o_test=o_test, y_test=y_test,
                     groups_test=groups_test)
    with open(os.path.join(base_dir, "apnea-ecg.pkl"), "wb") as f:
        pickle.dump(apnea_ecg, f, protocol=2)

    print("\nok!")

Training...
a17: 100%|██████████| 485/485 [50:04<00:00,  6.20s/it]

Testing...
x12: 100%|██████████| 527/527 [48:53<00:00,  5.57s/it]

ok!


In [ ]:
!pip install keras
!pip install tensorflow
#
#IMPORT LIBRARIES
import pickle
import numpy as np
import tensorflow
import os
from keras.callbacks import LearningRateScheduler,EarlyStopping
from keras.layers import  Dense,Flatten,MaxPooling2D,Conv2D,Permute,Reshape,LSTM,BatchNormalization,Bidirectional
from keras.regularizers import l2
from scipy.interpolate import splev, splrep
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
import keras
import matplotlib.pyplot as plt
from keras.layers import Conv1D, Dense, Dropout, Flatten, MaxPooling1D,Conv2D
from keras.models import Model, load_model, save_model
from keras.layers import Input
import pandas as pd
from keras.layers import GRU
from keras.layers import add, concatenate
from keras.layers import Reshape, Lambda

#base_dir = "dataset"
base_dir = "apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0"
#base_dir = "physionet.org/files/apnea-ecg/1.0.0"

#------------------------------------------------------------------------------
# DEEP LEARNING MODELS NEED TO VECTORS OR MATRIX WITH SAME SIZE
# R-R INTERVALS DONT HAVE SAME SIZE, SO WE NEED TO INTERPOLATE VECTORS TO GET VECTORS WITH SAME SIZE.
# BASED ON OUR EXPERIENCE INTERPOLATION IN 3 HZ BETTER AND ACCURATE.
ir = 3 # INTERPOLATION RATE(3HZ)
time_range= 60 # 60-s INTERVALS OF ECG SIGNALS
weight=1e-3 #  WEIGHT L2 FOR REGULARIZATION(AVODING OVERFITTING PARAMETER)
#-----------------------------
# NORMALIZATION:
# DEEP LEARNING AND EVEN NEURAL NETWORKS INPUT SHOULD BE NORMALIZED:
# MIN-MAX METHOD APPLIED FOR SCALING:(Array-min(Array))/(max(Array)-min(Array))
scaler = lambda arr: (arr - np.min(arr)) / (np.max(arr) - np.min(arr))
#-----------------------------
# FIRSTLY WE PRE-PROCESSED OUR DATA IN "apnea-ecg.pkl" FILE
# IN PRE-PROCESSING SECTION WE EXTRACT R-R INTERVALS AND R-PEAK AMPLITUDES
# IN THIS PART WE LOAD THIS DATA AND INTERPOLATE AND CONCATE FOR FEEDING TO NETWORKS
def load_data():
    tm = np.arange(0, (before + 1 + after) * 60, step=1 / float(ir))

    with open(os.path.join(base_dir, "apnea-ecg.pkl"), 'rb') as f: # read preprocessing result
        apnea_ecg = pickle.load(f)
#-----------------
    x_train = []
    o_train, y_train = apnea_ecg["o_train"], apnea_ecg["y_train"]

    for i in range(len(o_train)):
        (rri_tm, rri_signal), (amp_tm, amp_signal) = o_train[i]
      # Curve interpolation
        rri_interp_signal = splev(tm, splrep(rri_tm, scaler(rri_signal), k=3), ext=1)
        amp_interp_signal = splev(tm, splrep(amp_tm, scaler(amp_signal), k=3), ext=1)
        x_train.append([rri_interp_signal, amp_interp_signal])
    x_train = np.array(x_train, dtype="float32")

    x_train = np.expand_dims(x_train,1)
    x_train=np.array(x_train, dtype="float32").transpose((0,3,1,2)) # convert to numpy format

    #return x_train_final, y_train
    return x_train, y_train

In [ ]:
!pip install Hyperactive
!pip install mealpy

from keras.layers.rnn import GRU
from keras.layers import Reshape, Lambda
from keras.layers import GRU
from hyperactive import Hyperactive
from hyperactive.optimizers import HillClimbingOptimizer
from tensorflow.keras import optimizers
#from Hyperactive.hyperactive import RandomSearchOptimizer, MHoneyBadgerAlgorithm, HoneyBadgerAlgorithm
from keras.optimizers import adam
import threading
import concurrent.futures
from threading import Thread
import time
from tabnanny import verbose
from mealpy.swarm_based.HBA import OriginalHBA
from tensorflow.python.keras import optimizers
from keras.optimizers import adam
#from HBA import HBA
import warnings
warnings.filterwarnings('ignore')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 656.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.0/378.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 53.7 MB/s eta 0:00:00


In [ ]:
#print(x_train.shape, y_train.shape)
#x_train= np.reshape(x_train,(x_train.shape[0],450,2,2))    #90---450?
#x_train= np.concatenate((x_train[:,:450,:,0],x_train[:,:450,:,1]),axis=1)
#x_train= x_train.transpose((0,2,1))
#print(x_train.shape, y_train.shape)
##(16709, 900, 1, 2) (16709, 2)
##(16709, 2, 900) (16709, 2)


In [ ]:
# BHA
###########################
def fitness_function(position):
    learning_rate,n1,n2,n3,n4,n5,n6,n7,epochs = position[0],position[1],position[2],position[3],position[4],position[5],position[6],position[7],position[8]
    n1,n2,n3,n4,n5,n6,n7,epochs=int(n1),int(n2),int(n3),int(n4),int(n5),int(n6),int(n7),int(epochs)
    #return learning_rate,n1,n2,n3,n4,n5,n6,epochs

    print(learning_rate,n1,n2,n3,n4,n5,n6,n7,epochs)

#def create_model(weight=1e-3):
    model=Sequential()
    model.add(Reshape((450,2,2),input_shape=(900,1,2)))
    model.add(Conv2D(n1, kernel_size=(11,1), strides=(1,1), padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight),input_shape=(900,1,2)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3,1)))
    model.add(Conv2D(n2, kernel_size=(5,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3,1)))
    model.add(Conv2D(n3, kernel_size=(3,1), strides=(1,1), padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(BatchNormalization())
    model.add(Conv2D(n4, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(BatchNormalization())
    model.add(Conv2D(n5, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3,1),strides=(2,1)))

    model.add(Permute((2,1,3)))
    model.add(Reshape((2,24*n5)))

    model.add(GRU(n6, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(n7, activation="relu"))

    model.add(Dense(2, activation="softmax"))

    return learning_rate,n1,n2,n3,n4,n5,n6,n7,epochs

    optimizer =keras.optimizers.Adam(lr=learning_rate)
    print("Learning rate: ", learning_rate)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
    #model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=['accuracy'])

    history=model.fit(X1, Y1, batch_size=128, epochs=int(epochs), validation_data=(x_val, y_val),
                       callbacks=[callback1,lr_scheduler])


    #history = model.fit(x_train,y_train, epochs=int(epochs),batch_size=32, validation_data=(x_test, y_test))
    return history.history['val_accuracy'][-1]
    #loss, accuracy = model.evaluate(x_test, y_test)

    #return model
#------------------------------------------------------------------------------
# Define learning rate schedule for preventing overfitting in deep learning methods:
def lr_schedule(epochs, learning_rate):
   if epochs > 70 and \
           (epochs - 1) % 10 == 0:
        learning_rate *= 0.1
   print("Learning rate: ", learning_rate)
   return learning_rate

#------------------------------------------------------------------------------

        # we used k-fold cross-validation for more reliable experiments:
   kfold = StratifiedKFold(n_splits=5, shuffle=True,random_state=7)
   cvscores = []
   ACC=[]
   SN=[]
   SP=[]
   F2=[]
   print("train num:", len(y_train))
   print("test num:", len(y_test))


    # separate train& test and then compile model
  # for train, test in kfold.split(x_train, y_train.argmax(1)):
     #model = create_model()
     #model.summary()

     # Compile and evaluate model:
if __name__ == "__main__":
    # loading Data:
    x_train, y_train = load_data()
    # we have labels(Y) in a binary way 0 for normal and 1 for apnea patients
    # we want to classify data into 2-class so we changed y in a categorical way:
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
    # we used k-fold cross-validation for more reliable experiments:
    kfold = StratifiedKFold(n_splits=5, shuffle=True,random_state=7)


    # separate train& test and then compile model
    ##for train, test in kfold.split(x_train, y_train.argmax(1)):
     #model = create_model()
     #model.summary()

    # define callback for early stopping:
    lr_scheduler = LearningRateScheduler(lr_schedule)
    callback1 = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    #x_train= np.reshape(x_train,(x_train.shape[0],450,2,2))    #90---450?
    #x_train= np.concatenate((x_train[:,:450,:,0],x_train[:,:450,:,1]),axis=1)
    #x_train= x_train.transpose((0,2,1))
    #10% of Data used for validation:
    X1,x_val,Y1,y_val=train_test_split(x_train,y_train,test_size=0.10)

##################################################################
#pso = Pso(swarmsize=4,maxiter=14)
# n,sf,sp,l
##bp,value = pso.run(func,[1,2,2,2],[16,8,4,4])
#v = func(bp);
##################################################################
problem_dict1 = {
    "fit_func": fitness_function,
    'lb':[0.001,64,64,64,64,32,16,4,50], # Lower bound of our parameters
    'ub':[0.001,256,512,512,512,512,256,64,50], # upper bound of our parameters
    "minmax": "min",
    "obj_weights": [0.4, 1, 1,1,1,1,1,1,1]               # Define it or default value will be [1, 1, 1]   [0.4, 0.1, 0.5,1,1,1,1,1]
                }

rng = np.random.default_rng()
epoch = 50
pop_size =200

#def thread_function(k):

  #run=k+1
  #print("Run Number",run)
  #Optimizer = MHoneyBadgerAlgorithm(search_config, n_iter=100, n_part=10, metric='accuracy', cv=10, h_beta=6.0, h_c=2.0,run=run)
model = OriginalHBA(epoch, pop_size)
  #t1 = time.time()

  #Optimizer.fit(x_train, y_train)
 # t2 = time.time()

  #print("time: {}".format(t2-t1))



#best_position, best_fitness = model.solve(problem_dict1)
best_position, best_fitness= model.solve(problem_dict1)
#best_position, best_fitness= model.solve(mode="thread")
v = fitness_function(best_position);
#print(f"Solution: {best_position}, Fitness: {best_fitness}")
#return v
print(f"Solution: {best_position}, Fitness: {v}")

0.001 95 254 384 385 407 64 18 50


INFO:mealpy.swarm_based.HBA.OriginalHBA:Solving 9-objective optimization problem with weights: [0.4 1.  1.  1.  1.  1.  1.  1.  1. ].


0.001 92 256 391 378 405 64 21 50
0.001 87 258 386 385 491 64 26 50
0.001 88 247 384 375 348 64 8 50
0.001 88 260 381 377 182 64 23 50
0.001 91 257 389 384 391 64 11 50
0.001 92 253 383 374 485 64 15 50
0.001 93 257 374 387 317 64 9 50
0.001 95 260 379 389 137 64 21 50
0.001 94 265 376 392 383 64 9 50
0.001 91 255 375 374 445 64 12 50
0.001 100 256 393 374 368 64 12 50
0.001 94 255 375 388 309 64 18 50
0.001 93 259 384 380 420 64 19 50
0.001 93 252 374 379 509 64 11 50
0.001 92 247 388 390 156 64 9 50
0.001 102 252 390 386 501 64 14 50
0.001 95 263 384 377 281 64 17 50
0.001 92 258 393 390 165 64 27 50
0.001 99 257 385 374 407 64 12 50
0.001 93 263 387 376 498 64 26 50
0.001 87 260 374 390 394 64 22 50
0.001 99 260 388 389 507 64 16 50
0.001 86 246 385 387 182 64 18 50
0.001 90 252 374 377 289 64 14 50
0.001 101 251 381 389 252 64 14 50
0.001 87 254 389 385 414 64 8 50
0.001 98 264 383 379 278 64 11 50
0.001 99 251 379 393 287 64 8 50
0.001 94 256 391 378 196 64 15 50
0.001 101 246 381

INFO:mealpy.swarm_based.HBA.OriginalHBA:>Problem: P, Epoch: 1, Current best: 1336.0004, Global best: 1336.0004, Runtime: 81.40350 seconds


0.001 86 253 374 374 128 64 8 50
0.001 86 246 379 384 133 64 12 50
0.001 86 249 374 374 128 64 8 50
0.001 86 253 394 394 411 64 8 50
0.001 86 251 374 374 136 64 8 50
0.001 86 246 374 374 128 64 11 50
0.001 90 253 374 374 139 64 8 50
0.001 93 251 374 374 128 64 11 50
0.001 86 255 377 374 129 64 8 50
0.001 86 257 374 380 128 64 8 50
0.001 86 246 375 376 128 64 9 50
0.001 86 251 374 374 131 64 8 50
0.001 86 256 394 394 128 64 8 50
0.001 86 246 394 394 128 64 8 50
0.001 86 254 394 394 151 64 8 50
0.001 106 252 394 374 178 64 8 50
0.001 95 252 394 374 141 64 8 50
0.001 86 246 375 378 128 64 9 50
0.001 106 266 374 394 143 64 8 50
0.001 86 246 374 374 128 64 8 50
0.001 86 257 374 374 143 64 12 50
0.001 91 266 374 374 128 64 8 50
0.001 86 252 374 374 128 64 8 50
0.001 86 252 394 394 128 64 8 50
0.001 89 252 374 374 128 64 11 50
0.001 86 266 394 374 128 64 8 50
0.001 86 246 394 394 128 64 8 50
0.001 86 266 374 374 128 64 10 50
0.001 86 248 374 374 128 64 8 50
0.001 86 253 374 374 141 64 8 50
0.

...

0.001 106 266 394 394 128 64 10 50
0.001 106 246 374 394 128 64 28 50
0.001 86 246 374 374 128 64 8 50
0.001 86 246 394 374 128 64 11 50
0.001 86 246 374 374 128 64 8 50
0.001 86 246 374 374 128 64 8 50
0.001 103 266 374 374 154 64 9 50
0.001 106 246 394 394 184 64 11 50
0.001 86 246 374 374 128 64 8 50
0.001 86 246 374 374 128 64 8 50
0.001 86 246 374 374 128 64 8 50
0.001 86 246 374 374 128 64 8 50
0.001 86 246 374 374 128 64 8 50
0.001 86 246 376 374 128 64 8 50
0.001 106 246 374 374 128 64 8 50
0.001 86 246 394 394 512 64 8 50
0.001 86 246 374 374 128 64 8 50
0.001 86 266 374 374 128 64 8 50
0.001 86 266 394 394 128 64 9 50
0.001 86 266 394 374 512 64 28 50
0.001 86 246 374 394 128 64 28 50
0.001 86 246 374 374 128 64 8 50
0.001 95 246 394 374 142 64 8 50
0.001 106 266 394 394 128 64 10 50
0.001 86 246 374 374 128 64 8 50
0.001 86 246 374 394 164 64 10 50
0.001 106 266 394 394 160 64 10 50
0.001 86 246 374 374 128 64 8 50
0.001 86 246 374 374 128 64 8 50
0.001 86 246 374 374 128 64

INFO:mealpy.swarm_based.HBA.OriginalHBA:>Problem: P, Epoch: 49, Current best: 1330.0004, Global best: 1330.0004, Runtime: 79.31733 seconds


0.001 99 266 394 374 128 64 8 50
0.001 106 266 394 394 128 64 8 50
0.001 86 266 374 374 467 64 8 50
0.001 106 246 374 394 128 64 11 50
0.001 86 246 374 374 128 64 8 50
0.001 103 246 374 374 128 64 8 50
0.001 106 246 394 374 185 64 8 50
0.001 86 246 374 374 128 64 8 50
0.001 86 246 374 374 128 64 8 50
0.001 86 246 374 374 128 64 8 50
0.001 106 246 394 374 128 64 8 50
0.001 86 246 394 374 153 64 9 50
0.001 86 246 374 374 128 64 8 50
0.001 86 248 374 377 128 64 8 50
0.001 86 266 394 394 142 64 8 50
0.001 86 266 394 374 128 64 28 50
0.001 86 246 374 374 128 64 9 50
0.001 86 246 374 374 128 64 8 50
0.001 106 246 394 374 128 64 8 50
0.001 86 246 394 374 512 64 8 50
0.001 86 246 374 374 128 64 8 50
0.001 86 246 388 374 132 64 8 50
0.001 86 246 374 374 128 64 8 50
0.001 86 246 374 374 128 64 8 50
0.001 106 246 394 394 128 64 11 50
0.001 86 246 374 374 128 64 8 50
0.001 105 246 374 374 128 64 8 50
0.001 86 266 394 374 182 64 8 50
0.001 86 246 394 374 128 64 9 50
0.001 86 246 374 374 128 64 8 50

INFO:mealpy.swarm_based.HBA.OriginalHBA:>Problem: P, Epoch: 50, Current best: 1330.0004, Global best: 1330.0004, Runtime: 80.43814 seconds


0.001 86 246 374 374 128 64 8 50
Solution: [1.00e-03 8.60e+01 2.46e+02 3.74e+02 3.74e+02 1.28e+02 6.40e+01 8.00e+00
 5.00e+01], Fitness: (0.001, 86, 246, 374, 374, 128, 64, 8, 50)
